In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score, matthews_corrcoef, average_precision_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import roc_curve
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import StratifiedKFold
import math


import sys
sys.path.append('/home/ss2686/03_DICTrank')
import argparse
from scripts.evaluation_functions import evaluate_classifier, optimize_threshold_j_statistic


# Path where your data is stored
data_path = '../data/processed_binarised__splits/'

results = {}
held_out_results = []

def generate_fingerprints(smiles_list):
    fps = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, arr)
        fps.append(arr)
    return np.array(fps)


# Assuming PK dataset is regression and others are classification
for dataset in os.listdir(data_path):
    
    # Exclude hidden files or directories like .ipynb_checkpoints
    if dataset.startswith('.'):
        continue
    print(dataset)

    # Get all the file names for this dataset
    all_files = os.listdir(os.path.join(data_path, dataset))

    # Extract activity names by removing the _train.csv.gz or _test.csv.gz from file names
    activity_names = list(set([f.replace("_train.csv.gz", "").replace("_test.csv.gz", "")  for f in all_files if not f.startswith(".ipynb_checkpoints")]))

    for activity in tqdm(activity_names, desc="Processing activities"):
        
        train_path = os.path.join(data_path, dataset, f"{activity}_train.csv.gz")
        test_path = os.path.join(data_path, dataset, f"{activity}_test.csv.gz")

        train_df = pd.read_csv(train_path, compression='gzip')
        test_df = pd.read_csv(test_path, compression='gzip')

        X_train = generate_fingerprints(train_df['Standardized_SMILES'])
        X_test = generate_fingerprints(test_df['Standardized_SMILES'])
        y_train = train_df[activity]
        y_test = test_df[activity]

      
        # Classification
        model = RandomForestClassifier(n_jobs=40)
            
        # Hyperparameter Optimization
        param_dist_classification = {'max_depth': randint(10, 20),
                          'max_features': randint(40, 50),
                          'min_samples_leaf': randint(5, 15),
                          'min_samples_split': randint(5, 15),
                          'n_estimators':[200, 300, 400, 500, 600],
                          'bootstrap': [True, False],
                          'oob_score': [False],
                          'random_state': [42],
                          'criterion': ['gini', 'entropy'],
                          'n_jobs': [40],
                          'class_weight' : [None, 'balanced']
                         }
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)   
            
        classification_search = HalvingRandomSearchCV(
                model,
                param_dist_classification,
                factor=3,
                cv=inner_cv,
                random_state=42,
                verbose=1,
                n_jobs=40)
            
        classification_search.fit(X_train, y_train)
        best_model = classification_search.best_estimator_
            
        # Random Over-sampling and Threshold Optimization
        sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
            
        pipeline = Pipeline(steps=[('sampler', sampler), ('model', best_model)])
        pipeline.fit(X_train, y_train)
            
        # Predict using threshold-optimized model
        probs_test = pipeline.predict_proba(X_test)[:, 1]
            
        oof_predictions = np.zeros(X_train.shape[0])
        oof_probs = np.zeros(X_train.shape[0])

        cv_scores = []

        for train_idx, valid_idx in inner_cv.split(X_train, y_train):
            X_train_fold, y_train_fold = X_train[train_idx], y_train[train_idx]
            X_valid_fold, y_valid_fold = X_train[valid_idx], y_train[valid_idx]

            # Random Over-sampling
            X_resampled, y_resampled = sampler.fit_resample(X_train_fold, y_train_fold)

            # Train the model on the resampled data
            best_model.fit(X_resampled, y_resampled)

            # Store out-of-fold predictions
            oof_predictions[valid_idx] = best_model.predict(X_valid_fold)
            oof_probs[valid_idx] = best_model.predict_proba(X_valid_fold)[:, 1]

            # AUC for this fold
            fold_auc = roc_auc_score(y_valid_fold, oof_probs[valid_idx])
            cv_scores.append(fold_auc)

        # Optimize the threshold using out-of-fold predictions
        best_threshold = optimize_threshold_j_statistic(y_train, oof_probs)
        predictions_test = (probs_test >= best_threshold).astype(int)
        
        results[activity] = {
                'CV_AUC_mean': np.mean(cv_scores),
                'CV_AUC_std': np.std(cv_scores),
                **evaluate_classifier(y_test, predictions_test, probs_test)
            }
        
        held_out_data = {
            'Dataset': dataset,
            "Actviity": activity,
            'SMILES': test_df['Standardized_SMILES'],
            'True_Value': y_test,
            'Prediction': predictions_test,
            'Probability': probs_test,
            'Best_Threshold': best_threshold
        }
        
        held_out_results.append(pd.DataFrame(held_out_data))         
        # Save results at each step
        pd.DataFrame(results).T.to_csv('./structural_model_results.csv')
              

# Save results
results_df = pd.DataFrame(results).T.reset_index(drop=False)
results_df = results_df.rename(columns={'index': 'endpoint'})
results_df.to_csv('./structural_model_results.csv', index=False)

# Concatenate and save held-out test set results
pd.concat(held_out_results).to_csv('./structural_model_held_out_test_results.csv', index=False)
            
        


cardiotox_with_sider_inactives


Processing activities:   0%|                              | 0/1 [00:00<?, ?it/s]

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 20
max_resources_: 1167
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 58
n_resources: 20
Fitting 5 folds for each of 58 candidates, totalling 290 fits
----------
iter: 1
n_candidates: 20
n_resources: 60
Fitting 5 folds for each of 20 candidates, totalling 100 fits
----------
iter: 2
n_candidates: 7
n_resources: 180
Fitting 5 folds for each of 7 candidates, totalling 35 fits
----------
iter: 3
n_candidates: 3
n_resources: 540
Fitting 5 folds for each of 3 candidates, totalling 15 fits


Processing activities: 100%|██████████████████████| 1/1 [00:15<00:00, 15.32s/it]


cardiotox_with_sider_actives


...

Processing activities: 100%|██████████████████████| 1/1 [00:12<00:00, 12.44s/it]


DICTrank


Processing activities:   0%|                              | 0/1 [00:00<?, ?it/s]

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 20
max_resources_: 934
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 46
n_resources: 20
Fitting 5 folds for each of 46 candidates, totalling 230 fits
----------
iter: 1
n_candidates: 16
n_resources: 60
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 2
n_candidates: 6
n_resources: 180
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 540
Fitting 5 folds for each of 2 candidates, totalling 10 fits


Processing activities: 100%|██████████████████████| 1/1 [00:12<00:00, 12.58s/it]


In [2]:
df = pd.read_csv('./structural_model_held_out_test_results.csv')
df

,Dataset,Actviity,SMILES,True_Value,Prediction,Probability,Best_Threshold
0,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),CC(C(=O)[O-])c1ccc(-c2ccccc2)c(F)c1,1,1,0.551970,0.516084
1,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),CC(=O)[NH+]1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(...,1,0,0.450407,0.516084
2,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),O=C1CCC([NH+]2C(=O)c3ccccc3C2=O)C(=O)N1,1,1,0.595547,0.516084
3,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),C=CC1C[NH+]2CCC1CC2C(O)c1ccnc2ccc(OC)cc12,1,1,0.578742,0.516084
4,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),CC[NH+](CC)CCNC(=O)c1ccc(N)cc1,1,1,0.586031,0.516084
...,...,...,...,...,...,...,...
472,DICTrank,DICTrank,COC(=O)N=c1[n-]c2ccc(C(=O)c3ccccc3)cc2[n-]1,0,1,0.574130,0.565187
473,DICTrank,DICTrank,CC(C)(C(=O)[O-])c1ccc(C(O)CCC[NH+]2CCC(C(O)(c3...,0,1,0.693600,0.565187
474,DICTrank,DICTrank,CCOC(=O)[NH+]1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,0,0,0.542195,0.565187
475,DICTrank,DICTrank,CCCSc1ccc2[n-]c(=NC(=O)OC)[n-]c2c1,0,0,0.498470,0.565187


In [3]:
df = pd.read_csv('./structural_model_results.csv')
df

,endpoint,CV_AUC_mean,CV_AUC_std,Held_out_TP,Held_out_TN,Held_out_FP,Held_out_FN,Held_out_BA,Held_out_AUC,Held_out_MCC,Held_out_AUCPR,Held_out_Specificity,Held_out_Sensitivity,Held_out_F1
0,Cardiotox (with SIDER inactives),0.690843,0.032524,49.0,11.0,10.0,16.0,0.638828,0.767766,0.257019,0.901701,0.523810,0.753846,0.790323
1,Cardiotox (with SIDER actives),0.677243,0.025024,34.0,19.0,2.0,31.0,0.713919,0.821978,0.372572,0.921852,0.904762,0.523077,0.673267
2,Cardiotox (with SIDER all),0.674016,0.025888,49.0,15.0,6.0,16.0,0.734066,0.799267,0.418863,0.916765,0.714286,0.753846,0.816667
3,Cardiac disorders,0.682941,0.034613,60.0,24.0,16.0,33.0,0.622581,0.674194,0.227185,0.822359,0.600000,0.645161,0.710059
4,DICTrank,0.665871,0.057278,36.0,17.0,4.0,29.0,0.681685,0.800000,0.312973,0.908918,0.809524,0.553846,0.685714
